In [186]:
#Creating Baseline using Nearest Means Classifier

In [187]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestCentroid
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [188]:
import pandas as pd

# Load the data_train DataFrame
data_train = pd.read_csv('mushroom_train.csv', usecols = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14, 15])

# Calculate the Pearson correlation coefficient for all pairs of columns
corr_matrix = data_train.corr()

# Display the correlation matrix
print(corr_matrix)

              cap-diameter  stem-height  stem-width
cap-diameter      1.000000     0.423171    0.695804
stem-height       0.423171     1.000000    0.436069
stem-width        0.695804     0.436069    1.000000


In [189]:
#categorical_col = ['cap-shape', 'cap-surface', 'cap-color', 'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color', 'stem-color', 'has-ring', 'ring-type', 'habitat', 'season']
categorical_col = ['cap-shape', 'cap-surface', 'cap-color', 'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color', 'stem-color', 'has-ring', 'ring-type', 'habitat', 'season','class']
categorical_data_train = data_train[categorical_col]
encoded_data_train = pd.get_dummies(categorical_data_train, prefix = categorical_col, prefix_sep='_')

In [190]:
numerical_col_train = data_train[['cap-diameter', 'stem-height', 'stem-width']]
scaler = StandardScaler()
numerical_col_scaled_train = scaler.fit_transform(numerical_col_train)
numerical_data_train = pd.DataFrame(numerical_col_scaled_train, columns=numerical_col_train.columns)

In [191]:
final_data_train = pd.concat([numerical_data_train, encoded_data_train], axis = 1)

In [192]:
df_temp = pd.DataFrame({'zeros': [0] * 42748})
df_temp.loc[final_data_train.iloc[:,-2] == 1, 'zeros'] = 0
df_temp.loc[final_data_train.iloc[:,-1] == 1, 'zeros'] = 1

In [193]:
final_data_train = final_data_train.drop(final_data_train.columns[-2:], axis=1)

In [194]:
final_data_train_watch = pd.concat([final_data_train, df_temp], axis=1)

In [195]:
final_data_train_watch = pd.concat([final_data_train, df_temp], axis=1)
final_data_train_watch = final_data_train_watch.rename(columns={'<old_column_name>': 'label'})

In [196]:
new_data_train_watch = final_data_train_watch.reset_index()
#print(new_data_train_watch.columns)

In [197]:
# define the columns of interest
cols_of_interest = ['stem-width', 'stem-height', 'cap-diameter']

# calculate the correlation matrix
corr_matrix = final_data_train.corr(method='pearson')

# filter the correlation matrix to only include rows or columns with 'stem-width', 'stem-height', or 'cap-diameter'
corr_filtered = corr_matrix[(corr_matrix.index.isin(cols_of_interest)) | (corr_matrix.columns.isin(cols_of_interest))]

# create a copy of the filtered correlation matrix for sorting
corr_sorted = corr_filtered.abs().unstack().sort_values(ascending=False).reset_index()

# remove self-correlations (correlations of features with themselves)
corr_sorted = corr_sorted[corr_sorted['level_0'] != corr_sorted['level_1']]

# rename the columns of the sorted correlation matrix
corr_sorted.columns = ['feature_1', 'feature_2', 'correlation_coefficient']

# filter the sorted correlation matrix to only include rows or columns with 'stem-width', 'stem-height', or 'cap-diameter'
corr_sorted_filtered = corr_sorted[(corr_sorted['feature_1'].isin(cols_of_interest)) | (corr_sorted['feature_2'].isin(cols_of_interest))]

# display the top 5 positive and top 5 negative correlation coefficient values with their corresponding row and column names
print('Top 10 positive correlations:')
print(corr_sorted_filtered.head(10))

print('\nTop 10 negative correlations:')
print(corr_sorted_filtered.tail(10)[::-1])

Top 10 positive correlations:
            feature_1     feature_2  correlation_coefficient
3        cap-diameter    stem-width                 0.695804
4          stem-width  cap-diameter                 0.695804
5         stem-height    stem-width                 0.436069
6          stem-width   stem-height                 0.436069
7         stem-height  cap-diameter                 0.423171
8        cap-diameter   stem-height                 0.423171
9         ring-type_m   stem-height                 0.406376
10  gill-attachment_p    stem-width                 0.400621
11  gill-attachment_p  cap-diameter                 0.353896
12        ring-type_f   stem-height                 0.332793

Top 10 negative correlations:
          feature_1     feature_2  correlation_coefficient
275    gill-color_o    stem-width                 0.000822
274     cap-color_e  cap-diameter                 0.000841
273    stem-color_u   stem-height                 0.000941
272     ring-type_l    stem-widt

In [198]:
filtered_cols = final_data_train.columns

In [199]:
new_data_train_watch = pd.concat([new_data_train_watch, df_temp], axis=1)
#final_data_train = final_data_train.to_numpy()

In [200]:
# Reset the index of the DataFrame and create a new column 'index'
new_data_train_watch.reset_index(inplace=True)

# Rename the default index column to 'index'
new_data_train_watch.rename(columns={'index': 'index_column'}, inplace=True)
#print(new_data_train_watch)

In [201]:
# select the rows where 'ring-type_m' is 1
selected_rows_shape_m = new_data_train_watch[new_data_train_watch['ring-type_m'] == 1]

# create a new dataframe with the selected rows
selected_data_m = pd.DataFrame(selected_rows_shape_m, columns=new_data_train_watch.columns)

selected_data_m = new_data_train_watch.loc[new_data_train_watch['ring-type_m'] == 1].copy()

stem_width_min = selected_data_m['stem-width'].min()
stem_width_max = selected_data_m['stem-width'].max()
stem_width_mean = selected_data_m['stem-width'].mean()

selected_data_m['stem_width_ring_type_m_min'] = stem_width_min
selected_data_m['stem_width_ring_type_m_max'] = stem_width_max
selected_data_m['stem_width_ring_type_m_mean'] = stem_width_mean

# Rename the 'zeros' column to 'label1'
selected_data_m = selected_data_m.rename(columns={'zeros': 'label1'})

# Drop the duplicate 'label2' column if it exists
selected_data_m = selected_data_m.loc[:, ~selected_data_m.columns.duplicated()]

# Select the desired columns
selected_data_m = selected_data_m[['index_column', 'stem_width_ring_type_m_min', 'stem_width_ring_type_m_max', 'stem_width_ring_type_m_mean', 'label1']]

#print(selected_data_m.columns)

In [202]:
# select the rows where 'gill-attachment_p' is 1
selected_rows_shape_p = new_data_train_watch[new_data_train_watch['gill-attachment_p'] == 1]

# create a new dataframe with the selected rows
selected_data = pd.DataFrame(selected_rows_shape_p, columns=new_data_train_watch.columns)

selected_data_p = new_data_train_watch.loc[new_data_train_watch['gill-attachment_p'] == 1].copy()

stem_width_min = selected_data_p['stem-width'].min()
stem_width_max = selected_data_p['stem-width'].max()
stem_width_mean = selected_data_p['stem-width'].mean()

selected_data_p['stem_width_gill_attachment_p_min'] = stem_width_min
selected_data_p['stem_width_gill_attachment_p_max'] = stem_width_max
selected_data_p['stem_width_gill_attachment_p_mean'] = stem_width_mean

# Rename the 'zeros' column to 'label2'
selected_data_p = selected_data_p.rename(columns={'zeros': 'label2'})

print(selected_data_p)

# Drop the duplicate 'label2' column if it exists
selected_data_p = selected_data_p.loc[:, ~selected_data_p.columns.duplicated()]

# Select the desired columns
selected_data_p = selected_data_p[['index_column', 'stem_width_gill_attachment_p_min', 'stem_width_gill_attachment_p_max', 'stem_width_gill_attachment_p_mean', 'label2']]


       level_0  index_column  cap-diameter  stem-height  stem-width  \
4            4             4      0.160126     0.542345    0.631930   
22          22            22      2.066276     2.982750    2.244163   
46          46            46     -0.188536    -0.731298   -0.473544   
49          49            49      2.342141     4.122793    7.335741   
59          59            59     -0.544862    -1.123187   -0.763406   
...        ...           ...           ...          ...         ...   
42694    42694         42694      0.405340     0.364213    1.145686   
42695    42695         42695      0.089244    -0.259248   -0.032754   
42700    42700         42700      1.031783     1.106428    1.214653   
42710    42710         42710      0.064340    -0.606605   -0.048746   
42745    42745         42745      7.495450    -0.247373    1.423554   

       cap-shape_b  cap-shape_c  cap-shape_f  cap-shape_o  cap-shape_p  ...  \
4                0            0            0            0           

In [203]:
# select the rows where 'ring-type_f' is 1
selected_rows_shape_f = new_data_train_watch[new_data_train_watch['ring-type_f'] == 1]

# create a new dataframe with the selected rows
selected_data = pd.DataFrame(selected_rows_shape_f, columns=new_data_train_watch.columns)

selected_data_f = new_data_train_watch.loc[new_data_train_watch['ring-type_f'] == 1].copy()

stem_height_min = selected_data_f['stem-height'].min()
stem_height_max = selected_data_f['stem-height'].max()
stem_height_mean = selected_data_f['stem-height'].mean()

selected_data_f['stem_height_ring_type_f_min'] = stem_height_min
selected_data_f['stem_height_ring_type_f_max'] = stem_height_max
selected_data_f['stem_height_ring_type_f_mean'] = stem_height_mean

# Rename the 'zeros' column to 'label3'
selected_data_f = selected_data_f.rename(columns={'zeros': 'label3'})

# Drop the duplicate 'label3' column if it exists
selected_data_f = selected_data_f.loc[:, ~selected_data_f.columns.duplicated()]

# Select the desired columns
selected_data_f = selected_data_f[['index_column', 'stem_height_ring_type_f_min', 'stem_height_ring_type_f_max', 'stem_height_ring_type_f_mean', 'label3']]

#print(selected_data_f.columns)

In [204]:
# select the rows where 'gill-color_o' is 1
selected_rows_shape_o = new_data_train_watch[new_data_train_watch['gill-color_o'] == 1]

# create a new dataframe with the selected rows
selected_data = pd.DataFrame(selected_rows_shape_o, columns=new_data_train_watch.columns)

selected_data_o = new_data_train_watch.loc[new_data_train_watch['gill-color_o'] == 1].copy()

stem_width_min = selected_data_o['stem-width'].min()
stem_width_max = selected_data_o['stem-width'].max()
stem_width_mean = selected_data_o['stem-width'].mean()

selected_data_o['stem_width_gill_color_o_min'] = stem_width_min
selected_data_o['stem_width_gill_color_o_max'] = stem_width_max
selected_data_o['stem_width_gill_color_o_mean'] = stem_width_mean

# Rename the 'zeros' column to 'label4'
selected_data_o = selected_data_o.rename(columns={'zeros': 'label4'})

# Drop the duplicate 'label4' column if it exists
selected_data_o = selected_data_o.loc[:, ~selected_data_o.columns.duplicated()]

# Select the desired columns
selected_data_o = selected_data_o[['index_column', 'stem_width_gill_color_o_min', 'stem_width_gill_color_o_max', 'stem_width_gill_color_o_mean', 'label4']]

print(selected_data_o.columns)

Index(['index_column', 'stem_width_gill_color_o_min',
       'stem_width_gill_color_o_max', 'stem_width_gill_color_o_mean',
       'label4'],
      dtype='object')


In [205]:
# select the rows where 'cap-color_e' is 1
selected_rows_shape_e = new_data_train_watch[new_data_train_watch['cap-color_e'] == 1]

# create a new dataframe with the selected rows
selected_data = pd.DataFrame(selected_rows_shape_e, columns=new_data_train_watch.columns)

selected_data_e = new_data_train_watch.loc[new_data_train_watch['cap-color_e'] == 1].copy()

cap_diameter_min = selected_data_e['cap-diameter'].min()
cap_diameter_max = selected_data_e['cap-diameter'].max()
cap_diameter_mean = selected_data_e['cap-diameter'].mean()

selected_data_e['cap_diameter_cap_color_e_min'] = cap_diameter_min
selected_data_e['cap_diameter_cap_color_e_max'] = cap_diameter_max
selected_data_e['cap_diameter_cap_color_e_mean'] = cap_diameter_mean

# Rename the 'zeros' column to 'label5'
selected_data_e = selected_data_e.rename(columns={'zeros': 'label5'})

# Drop the duplicate 'label5' column if it exists
selected_data_e = selected_data_e.loc[:, ~selected_data_e.columns.duplicated()]

# Select the desired columns
selected_data_e = selected_data_e[['index_column', 'cap_diameter_cap_color_e_min', 'cap_diameter_cap_color_e_max', 'cap_diameter_cap_color_e_mean', 'label5']]

print(selected_data_e.columns)

Index(['index_column', 'cap_diameter_cap_color_e_min',
       'cap_diameter_cap_color_e_max', 'cap_diameter_cap_color_e_mean',
       'label5'],
      dtype='object')


In [206]:
# select the rows where 'cap-shape_p' is 1
selected_rows_shape_p_only = new_data_train_watch[new_data_train_watch['cap-shape_p'] == 1]

# create a new dataframe with the selected rows
selected_data = pd.DataFrame(selected_rows_shape_p_only, columns=new_data_train_watch.columns)

selected_data_p_only = new_data_train_watch.loc[new_data_train_watch['cap-shape_p'] == 1].copy()

cap_diameter_min = selected_data_p_only['cap-diameter'].min()
cap_diameter_max = selected_data_p_only['cap-diameter'].max()
cap_diameter_mean = selected_data_p_only['cap-diameter'].mean()

selected_data_p_only['cap_diameter_p_min'] = cap_diameter_min
selected_data_p_only['cap_diameter_p_max'] = cap_diameter_max
selected_data_p_only['cap_diameter_p_mean'] = cap_diameter_mean

# Rename the 'zeros' column to 'label6'
selected_data_p_only = selected_data_p_only.rename(columns={'zeros': 'label6'})

# Drop the duplicate 'label6' column if it exists
selected_data_p_only = selected_data_p_only.loc[:, ~selected_data_p_only.columns.duplicated()]

# Select the desired columns
selected_data_p_only = selected_data_p_only[['index_column', 'cap_diameter_p_min', 'cap_diameter_p_max', 'cap_diameter_p_mean', 'label6']]

print(selected_data_p_only.columns)

Index(['index_column', 'cap_diameter_p_min', 'cap_diameter_p_max',
       'cap_diameter_p_mean', 'label6'],
      dtype='object')


In [207]:
import pandas as pd

# Create a list of dataframes to concatenate
dataframes = [new_data_train_watch, selected_data_m, selected_data_p, selected_data_f, selected_data_o, selected_data_e, selected_data_p_only]


# Merge dataframes on a common index column
concatenated_df = pd.concat(dataframes, axis=1, join='outer')

# Create a common index column
concatenated_df['common_index'] = concatenated_df.index

# Replace NaN values with '0'
concatenated_df.fillna(0, inplace=True)

# Set the common index as the new index
concatenated_df.set_index('common_index', inplace=True)

# Print the column names of the concatenated dataframe
print(concatenated_df.columns)

Index(['level_0', 'index_column', 'cap-diameter', 'stem-height', 'stem-width',
       'cap-shape_b', 'cap-shape_c', 'cap-shape_f', 'cap-shape_o',
       'cap-shape_p',
       ...
       'index_column', 'cap_diameter_cap_color_e_min',
       'cap_diameter_cap_color_e_max', 'cap_diameter_cap_color_e_mean',
       'label5', 'index_column', 'cap_diameter_p_min', 'cap_diameter_p_max',
       'cap_diameter_p_mean', 'label6'],
      dtype='object', length=126)


In [208]:
# Assuming concatenated_df is your DataFrame
#concatenated_df = concatenated_df.drop('index', axis=1)
concatenated_df = concatenated_df.drop('index_column', axis=1)
concatenated_df = concatenated_df.drop('level_0', axis=1)
print(concatenated_df.columns)

Index(['cap-diameter', 'stem-height', 'stem-width', 'cap-shape_b',
       'cap-shape_c', 'cap-shape_f', 'cap-shape_o', 'cap-shape_p',
       'cap-shape_s', 'cap-shape_x',
       ...
       'stem_width_gill_color_o_mean', 'label4',
       'cap_diameter_cap_color_e_min', 'cap_diameter_cap_color_e_max',
       'cap_diameter_cap_color_e_mean', 'label5', 'cap_diameter_p_min',
       'cap_diameter_p_max', 'cap_diameter_p_mean', 'label6'],
      dtype='object', length=118)


In [209]:
# Define a function to check if any of the label columns has '1'
def has_label(row):
    label_cols = ['label1', 'label2', 'label3', 'label4', 'label5', 'label6']
    for col in label_cols:
        if row[col] == 1:
            return 1
    return 0

# Create the new 'label' column by applying the 'has_label' function to each row
concatenated_df['label'] = concatenated_df.apply(has_label, axis=1)
concatenated_df = concatenated_df.drop(['label1', 'label2', 'label3', 'label4','label5','label6'], axis=1)

# Print the updated concatenated_data
#print(concatenated_data)
print(concatenated_df.columns)

Index(['cap-diameter', 'stem-height', 'stem-width', 'cap-shape_b',
       'cap-shape_c', 'cap-shape_f', 'cap-shape_o', 'cap-shape_p',
       'cap-shape_s', 'cap-shape_x',
       ...
       'stem_width_gill_color_o_min', 'stem_width_gill_color_o_max',
       'stem_width_gill_color_o_mean', 'cap_diameter_cap_color_e_min',
       'cap_diameter_cap_color_e_max', 'cap_diameter_cap_color_e_mean',
       'cap_diameter_p_min', 'cap_diameter_p_max', 'cap_diameter_p_mean',
       'label'],
      dtype='object', length=113)


In [210]:
#shuffled_data = final_data_train_new.sample(frac=1, random_state=42)
#print(shuffled_data)
print(concatenated_df.shape)
concatenated_df = concatenated_df.drop(['zeros'], axis=1)
concatenated_df_train = concatenated_df

(42748, 113)


In [211]:
#shuffled_data_train = shuffled_data.iloc[:, :-1]
#label_train = shuffled_data.iloc[:, -1]
#shuffled_data_train = shuffled_data_train.drop(['label1', 'label2', 'label3', 'label4', 'label5', 'label6'], axis=1)

# Apply PCA to the shuffled data
pca = PCA(n_components=50)
pcs = pca.fit_transform(concatenated_df_train)

# Get the names of the original features with the highest absolute loadings for each component
num_top_features = 50  # change this to the desired number of top features
top_feature_names = []
for i in range(pca.n_components_):
    component_loadings = abs(pca.components_[i])
    sorted_indices = component_loadings.argsort()[::-1]
    top_feature_indices = sorted_indices[:num_top_features]
    top_feature_names.append(concatenated_df_train.columns[top_feature_indices].tolist())

# Concatenate the lists of top feature names into a single list
all_top_feature_names = [name for sublist in top_feature_names for name in sublist]

# Count the frequency of each feature and sort them in descending order
feature_counts = pd.Series(all_top_feature_names).value_counts().sort_values(ascending=False)

# Get the top 10 most frequent features
top_features = feature_counts.index[:50].tolist()

# Print the list of top feature names
print(top_features)
label_column_train = concatenated_df_train.iloc[:, -1]

['cap-shape_s', 'gill-color_n', 'habitat_g', 'gill-attachment_x', 'gill-color_y', 'cap-shape_x', 'cap-surface_d', 'cap-color_y', 'cap-surface_y', 'cap-shape_b', 'gill-attachment_d', 'cap-shape_f', 'gill-color_w', 'cap-surface_s', 'cap-color_w', 'stem-color_y', 'gill-attachment_s', 'cap-shape_o', 'label', 'cap-surface_g', 'cap-color_n', 'stem-color_n', 'habitat_d', 'cap-surface_h', 'stem-color_w', 'gill-color_p', 'gill-attachment_a', 'cap-color_o', 'stem-width', 'gill-spacing_d', 'gill-attachment_e', 'gill-color_g', 'cap-color_g', 'habitat_m', 'season_w', 'does-bruise-or-bleed_t', 'does-bruise-or-bleed_f', 'season_u', 'stem-color_g', 'habitat_l', 'season_s', 'has-ring_f', 'has-ring_t', 'cap-surface_e', 'ring-type_e', 'season_a', 'cap-surface_i', 'ring-type_z', 'cap-surface_k', 'cap-surface_t']


In [212]:
# Get the indices of the columns in shuffled_data that correspond to the top 70 features
top_feature_indices = [concatenated_df_train.columns.get_loc(feature) for feature in top_features[:70]]

# Create a new dataframe that contains only the top 70 features
top_70_features_data_train = concatenated_df_train.iloc[:, top_feature_indices]

# Print the shape of the new dataframe
#print(top_70_features_data.shape)

In [213]:
top_70_features_data_train

,cap-shape_s,gill-color_n,habitat_g,gill-attachment_x,gill-color_y,cap-shape_x,cap-surface_d,cap-color_y,cap-surface_y,cap-shape_b,...,season_s,has-ring_f,has-ring_t,cap-surface_e,ring-type_e,season_a,cap-surface_i,ring-type_z,cap-surface_k,cap-surface_t
common_index,,,,,,,,,,,,,,,,,,,,,
0,0,1,0,0,0,0,0,1,0,0,...,0,1,0,0,0,1,1,0,0,0
1,0,0,0,0,0,1,0,1,1,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,1,1,0,...,0,0,1,0,1,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1
4,0,0,0,0,1,1,1,0,0,0,...,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42743,0,1,0,0,0,0,0,0,1,0,...,0,0,1,0,1,0,0,0,0,0
42744,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
42745,0,0,0,0,1,0,0,1,1,0,...,1,1,0,0,0,0,0,0,0,0


In [214]:
data_test = pd.read_csv('mushroom_test.csv', usecols = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])
data_new_test = data_test.to_numpy()

# Calculate the Pearson correlation coefficient for all pairs of columns
corr_matrix = data_test.corr()

# Display the correlation matrix
print(corr_matrix)

#categorical_col = ['cap-shape', 'cap-surface', 'cap-color', 'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color', 'stem-color', 'has-ring', 'ring-type', 'habitat', 'season']
categorical_col = ['cap-shape', 'cap-surface', 'cap-color', 'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color', 'stem-color', 'has-ring', 'ring-type', 'habitat', 'season','class']
categorical_data_test = data_test[categorical_col]
encoded_data_test = pd.get_dummies(categorical_data_test, prefix = categorical_col, prefix_sep='_')
print(encoded_data_test.columns.tolist())

numerical_col_test = data_test[['cap-diameter', 'stem-height', 'stem-width']]
scaler = StandardScaler()
numerical_col_scaled_test = scaler.fit_transform(numerical_col_test)
numerical_data_test = pd.DataFrame(numerical_col_scaled_test, columns=numerical_col_test.columns)

final_data_test = pd.concat([numerical_data_test, encoded_data_test], axis = 1)
print(final_data_test.columns.tolist())

              cap-diameter  stem-height  stem-width
cap-diameter      1.000000     0.421315    0.694313
stem-height       0.421315     1.000000    0.436266
stem-width        0.694313     0.436266    1.000000
['cap-shape_b', 'cap-shape_c', 'cap-shape_f', 'cap-shape_o', 'cap-shape_p', 'cap-shape_s', 'cap-shape_x', 'cap-surface_d', 'cap-surface_e', 'cap-surface_g', 'cap-surface_h', 'cap-surface_i', 'cap-surface_k', 'cap-surface_l', 'cap-surface_s', 'cap-surface_t', 'cap-surface_w', 'cap-surface_y', 'cap-color_b', 'cap-color_e', 'cap-color_g', 'cap-color_k', 'cap-color_l', 'cap-color_n', 'cap-color_o', 'cap-color_p', 'cap-color_r', 'cap-color_u', 'cap-color_w', 'cap-color_y', 'does-bruise-or-bleed_f', 'does-bruise-or-bleed_t', 'gill-attachment_a', 'gill-attachment_d', 'gill-attachment_e', 'gill-attachment_f', 'gill-attachment_p', 'gill-attachment_s', 'gill-attachment_x', 'gill-spacing_c', 'gill-spacing_d', 'gill-spacing_f', 'gill-color_b', 'gill-color_e', 'gill-color_f', 'gill-color_g', 'g

In [215]:


###########

df_temp = pd.DataFrame({'zeros': [0] * (final_data_test.shape[0])})
df_temp.loc[final_data_test.iloc[:,-2] == 1, 'zeros'] = 0
df_temp.loc[final_data_test.iloc[:,-1] == 1, 'zeros'] = 1

final_data_test = final_data_test.drop(final_data_test.columns[-2:], axis=1)
#print(final_data_test.columns.tolist())

final_data_test_watch = pd.concat([final_data_test, df_temp], axis=1)
final_data_test_watch = final_data_test_watch.rename(columns={'<old_column_name>': 'label'})


# Reset the index of the DataFrame and create a new column 'index'
new_data_test_watch= final_data_test.reset_index()

# Rename the default index column to 'index'
new_data_test_watch.rename(columns={'index': 'index_column'}, inplace=True)
#print(new_data_train_watch)

new_data_test_watch = pd.concat([new_data_test_watch, df_temp], axis=1)

# Reset the index of the DataFrame and create a new column 'index'
new_data_test_watch.reset_index(inplace=True)

# Rename the default index column to 'index'
new_data_test_watch.rename(columns={'index': 'index_column'}, inplace=True)
print(new_data_test_watch.columns)

Index(['index_column', 'index_column', 'cap-diameter', 'stem-height',
       'stem-width', 'cap-shape_b', 'cap-shape_c', 'cap-shape_f',
       'cap-shape_o', 'cap-shape_p', 'cap-shape_s', 'cap-shape_x',
       'cap-surface_d', 'cap-surface_e', 'cap-surface_g', 'cap-surface_h',
       'cap-surface_i', 'cap-surface_k', 'cap-surface_l', 'cap-surface_s',
       'cap-surface_t', 'cap-surface_w', 'cap-surface_y', 'cap-color_b',
       'cap-color_e', 'cap-color_g', 'cap-color_k', 'cap-color_l',
       'cap-color_n', 'cap-color_o', 'cap-color_p', 'cap-color_r',
       'cap-color_u', 'cap-color_w', 'cap-color_y', 'does-bruise-or-bleed_f',
       'does-bruise-or-bleed_t', 'gill-attachment_a', 'gill-attachment_d',
       'gill-attachment_e', 'gill-attachment_f', 'gill-attachment_p',
       'gill-attachment_s', 'gill-attachment_x', 'gill-spacing_c',
       'gill-spacing_d', 'gill-spacing_f', 'gill-color_b', 'gill-color_e',
       'gill-color_f', 'gill-color_g', 'gill-color_k', 'gill-color_n',
    

In [216]:

# select the rows where 'ring-type_m' is 1
selected_rows_shape_m_test = new_data_test_watch[new_data_test_watch['ring-type_m'] == 1]

# create a new dataframe with the selected rows
selected_data_m_test = pd.DataFrame(selected_rows_shape_m_test, columns=new_data_test_watch.columns)

selected_data_m_test = new_data_test_watch.loc[new_data_test_watch['ring-type_m'] == 1].copy()

stem_width_min_test = selected_data_m_test['stem-width'].min()
stem_width_max_test = selected_data_m_test['stem-width'].max()
stem_width_mean_test = selected_data_m_test['stem-width'].mean()

selected_data_m_test['stem_width_ring_type_m_min'] = stem_width_min_test
selected_data_m_test['stem_width_ring_type_m_max'] = stem_width_max_test
selected_data_m_test['stem_width_ring_type_m_mean'] = stem_width_mean_test

# Rename the 'zeros' column to 'label1'
selected_data_m_test = selected_data_m_test.rename(columns={'zeros': 'label1'})

# Drop the duplicate 'label2' column if it exists
selected_data_m_test = selected_data_m_test.loc[:, ~selected_data_m_test.columns.duplicated()]

# Select the desired columns
selected_data_m_test = selected_data_m_test[['index_column', 'stem_width_ring_type_m_min', 'stem_width_ring_type_m_max', 'stem_width_ring_type_m_mean', 'label1']]

print(selected_data_m_test.columns)


Index(['index_column', 'stem_width_ring_type_m_min',
       'stem_width_ring_type_m_max', 'stem_width_ring_type_m_mean', 'label1'],
      dtype='object')


In [217]:
# select the rows where 'gill-attachment_p' is 1
selected_rows_shape_p_test = new_data_test_watch[new_data_test_watch['gill-attachment_p'] == 1]

# create a new dataframe with the selected rows
selected_data_test = pd.DataFrame(selected_rows_shape_p_test, columns=new_data_test_watch.columns)

selected_data_p_test = new_data_test_watch.loc[new_data_test_watch['gill-attachment_p'] == 1].copy()

stem_width_min_test = selected_data_p_test['stem-width'].min()
stem_width_max_test = selected_data_p_test['stem-width'].max()
stem_width_mean_test = selected_data_p_test['stem-width'].mean()

selected_data_p_test['stem_width_gill_attachment_p_min'] = stem_width_min
selected_data_p_test['stem_width_gill_attachment_p_max'] = stem_width_max
selected_data_p_test['stem_width_gill_attachment_p_mean'] = stem_width_mean

# Rename the 'zeros' column to 'label2'
selected_data_p_test = selected_data_p_test.rename(columns={'zeros': 'label2'})

# Drop the duplicate 'label2' column if it exists
selected_data_p_test = selected_data_p_test.loc[:, ~selected_data_p_test.columns.duplicated()]

# Select the desired columns
selected_data_p_test = selected_data_p_test[['index_column', 'stem_width_gill_attachment_p_min', 'stem_width_gill_attachment_p_max', 'stem_width_gill_attachment_p_mean', 'label2']]
print(selected_data_p_test.columns)

Index(['index_column', 'stem_width_gill_attachment_p_min',
       'stem_width_gill_attachment_p_max', 'stem_width_gill_attachment_p_mean',
       'label2'],
      dtype='object')


In [218]:
# select the rows where 'ring-type_f' is 1
selected_rows_shape_f_test = new_data_test_watch[new_data_test_watch['ring-type_f'] == 1]

# create a new dataframe with the selected rows
selected_data_test = pd.DataFrame(selected_rows_shape_f_test, columns=new_data_test_watch.columns)

selected_data_f_test = new_data_test_watch.loc[new_data_test_watch['ring-type_f'] == 1].copy()

stem_height_min_test = selected_data_f_test['stem-height'].min()
stem_height_max_test = selected_data_f_test['stem-height'].max()
stem_height_mean_test = selected_data_f_test['stem-height'].mean()

selected_data_f_test['stem_height_ring_type_f_min'] = stem_height_min_test
selected_data_f_test['stem_height_ring_type_f_max'] = stem_height_max_test
selected_data_f_test['stem_height_ring_type_f_mean'] = stem_height_mean_test

# Rename the 'zeros' column to 'label3'
selected_data_f_test = selected_data_f_test.rename(columns={'zeros': 'label3'})

# Drop the duplicate 'label3' column if it exists
selected_data_f_test = selected_data_f_test.loc[:, ~selected_data_f_test.columns.duplicated()]

# Select the desired columns
selected_data_f_test = selected_data_f_test[['index_column', 'stem_height_ring_type_f_min', 'stem_height_ring_type_f_max', 'stem_height_ring_type_f_mean', 'label3']]

print(selected_data_f_test.shape)

(15251, 5)


In [219]:
# select the rows where 'gill-color_o' is 1
selected_rows_shape_o_test = new_data_test_watch[new_data_test_watch['gill-color_o'] == 1]

# create a new dataframe with the selected rows
selected_data_test = pd.DataFrame(selected_rows_shape_o_test, columns=new_data_test_watch.columns)

selected_data_o_test = new_data_test_watch.loc[new_data_test_watch['gill-color_o'] == 1].copy()

stem_width_min_test = selected_data_o_test['stem-width'].min()
stem_width_max_test = selected_data_o_test['stem-width'].max()
stem_width_mean_test = selected_data_o_test['stem-width'].mean()

selected_data_o_test['stem_width_gill_color_o_min'] = stem_width_min_test
selected_data_o_test['stem_width_gill_color_o_max'] = stem_width_max_test
selected_data_o_test['stem_width_gill_color_o_mean'] = stem_width_mean_test

# Rename the 'zeros' column to 'label4'
selected_data_o_test = selected_data_o_test.rename(columns={'zeros': 'label4'})

# Drop the duplicate 'label4' column if it exists
selected_data_o_test = selected_data_o_test.loc[:, ~selected_data_o_test.columns.duplicated()]

# Select the desired columns
selected_data_o_test = selected_data_o_test[['index_column', 'stem_width_gill_color_o_min', 'stem_width_gill_color_o_max', 'stem_width_gill_color_o_mean', 'label4']]

print(selected_data_o_test.columns)

Index(['index_column', 'stem_width_gill_color_o_min',
       'stem_width_gill_color_o_max', 'stem_width_gill_color_o_mean',
       'label4'],
      dtype='object')


In [220]:
# select the rows where 'cap-color_e' is 1
selected_rows_shape_e_test = new_data_test_watch[new_data_test_watch['cap-color_e'] == 1]

# create a new dataframe with the selected rows
selected_data_test = pd.DataFrame(selected_rows_shape_e_test, columns=new_data_test_watch.columns)

selected_data_e_test = new_data_test_watch.loc[new_data_test_watch['cap-color_e'] == 1].copy()

cap_diameter_min_test = selected_data_e_test['cap-diameter'].min()
cap_diameter_max_test = selected_data_e_test['cap-diameter'].max()
cap_diameter_mean_test = selected_data_e_test['cap-diameter'].mean()

selected_data_e_test['cap_diameter_cap_color_e_min'] = cap_diameter_min_test
selected_data_e_test['cap_diameter_cap_color_e_max'] = cap_diameter_max_test
selected_data_e_test['cap_diameter_cap_color_e_mean'] = cap_diameter_mean_test

# Rename the 'zeros' column to 'label5'
selected_data_e_test = selected_data_e_test.rename(columns={'zeros': 'label5'})

# Drop the duplicate 'label5' column if it exists
selected_data_e_test = selected_data_e_test.loc[:, ~selected_data_e_test.columns.duplicated()]

# Select the desired columns
selected_data_e_test = selected_data_e_test[['index_column', 'cap_diameter_cap_color_e_min', 'cap_diameter_cap_color_e_max', 'cap_diameter_cap_color_e_mean', 'label5']]

print(selected_data_e_test.columns)

Index(['index_column', 'cap_diameter_cap_color_e_min',
       'cap_diameter_cap_color_e_max', 'cap_diameter_cap_color_e_mean',
       'label5'],
      dtype='object')


In [221]:
# select the rows where 'cap-shape_p' is 1
selected_rows_shape_p_only_test = new_data_test_watch[new_data_test_watch['cap-shape_p'] == 1]

# create a new dataframe with the selected rows
selected_data_test = pd.DataFrame(selected_rows_shape_p_only_test, columns=new_data_test_watch.columns)

selected_data_p_only_test = new_data_test_watch.loc[new_data_test_watch['cap-shape_p'] == 1].copy()

cap_diameter_min_test = selected_data_p_only_test['cap-diameter'].min()
cap_diameter_max_test = selected_data_p_only_test['cap-diameter'].max()
cap_diameter_mean_test = selected_data_p_only_test['cap-diameter'].mean()

selected_data_p_only_test['cap_diameter_p_min'] = cap_diameter_min_test
selected_data_p_only_test['cap_diameter_p_max'] = cap_diameter_max_test
selected_data_p_only_test['cap_diameter_p_mean'] = cap_diameter_mean_test

# Rename the 'zeros' column to 'label6'
selected_data_p_only_test = selected_data_p_only_test.rename(columns={'zeros': 'label6'})

# Drop the duplicate 'label6' column if it exists
selected_data_p_only_test = selected_data_p_only_test.loc[:, ~selected_data_p_only_test.columns.duplicated()]

# Select the desired columns
selected_data_p_only_test = selected_data_p_only_test[['index_column', 'cap_diameter_p_min', 'cap_diameter_p_max', 'cap_diameter_p_mean', 'label6']]

print(selected_data_p_only_test.columns)

Index(['index_column', 'cap_diameter_p_min', 'cap_diameter_p_max',
       'cap_diameter_p_mean', 'label6'],
      dtype='object')


In [222]:
import pandas as pd

# Create a list of dataframes to concatenate
dataframes_test = [new_data_test_watch, selected_data_m_test, selected_data_p_test, selected_data_f_test, selected_data_o_test, selected_data_e_test, selected_data_p_only_test]


# Merge dataframes on a common index column
concatenated_df_test = pd.concat(dataframes_test, axis=1, join='outer')

print(concatenated_df_test.columns)
column_name = "season_s"

if column_name in concatenated_df_test.columns:
    print("The column is present in the DataFrame.")
else:
    print("The column is not present in the DataFrame.")

Index(['index_column', 'index_column', 'cap-diameter', 'stem-height',
       'stem-width', 'cap-shape_b', 'cap-shape_c', 'cap-shape_f',
       'cap-shape_o', 'cap-shape_p',
       ...
       'index_column', 'cap_diameter_cap_color_e_min',
       'cap_diameter_cap_color_e_max', 'cap_diameter_cap_color_e_mean',
       'label5', 'index_column', 'cap_diameter_p_min', 'cap_diameter_p_max',
       'cap_diameter_p_mean', 'label6'],
      dtype='object', length=125)
The column is present in the DataFrame.


In [223]:
concatenated_df_test = concatenated_df_test.drop('index_column', axis=1)
print(concatenated_df_test.columns)

Index(['cap-diameter', 'stem-height', 'stem-width', 'cap-shape_b',
       'cap-shape_c', 'cap-shape_f', 'cap-shape_o', 'cap-shape_p',
       'cap-shape_s', 'cap-shape_x',
       ...
       'stem_width_gill_color_o_mean', 'label4',
       'cap_diameter_cap_color_e_min', 'cap_diameter_cap_color_e_max',
       'cap_diameter_cap_color_e_mean', 'label5', 'cap_diameter_p_min',
       'cap_diameter_p_max', 'cap_diameter_p_mean', 'label6'],
      dtype='object', length=117)


In [224]:
# Create a common index column
concatenated_df_test['common_index'] = concatenated_df_test.index

# Replace NaN values with '0'
concatenated_df_test.fillna(0, inplace=True)

# Set the common index as the new index
concatenated_df_test.set_index('common_index', inplace=True)

# Print the column names of the concatenated dataframe
print(concatenated_df_test.columns)

# Assuming concatenated_df is your DataFrame
#concatenated_df = concatenated_df.drop('index', axis=1)

print(concatenated_df_test.columns)

# Define a function to check if any of the label columns has '1'
def has_label(row):
    label_cols = ['label1', 'label2', 'label3', 'label4', 'label5', 'label6']
    for col in label_cols:
        if row[col] == 1:
            return 1
    return 0

# Create the new 'label' column by applying the 'has_label' function to each row
concatenated_df_test['label'] = concatenated_df_test.apply(has_label, axis=1)
concatenated_df_test = concatenated_df_test.drop(['label1', 'label2', 'label3', 'label4','label5','label6'], axis=1)
concatenated_df_test = concatenated_df_test.drop(['zeros'], axis=1)
# Print the updated concatenated_data
#print(concatenated_data)
print(concatenated_df_test.columns)
column_name = "season_s"

if column_name in concatenated_df_test.columns:
    print("The column is present in the DataFrame.")
else:
    print("The column is not present in the DataFrame.")

Index(['cap-diameter', 'stem-height', 'stem-width', 'cap-shape_b',
       'cap-shape_c', 'cap-shape_f', 'cap-shape_o', 'cap-shape_p',
       'cap-shape_s', 'cap-shape_x',
       ...
       'stem_width_gill_color_o_mean', 'label4',
       'cap_diameter_cap_color_e_min', 'cap_diameter_cap_color_e_max',
       'cap_diameter_cap_color_e_mean', 'label5', 'cap_diameter_p_min',
       'cap_diameter_p_max', 'cap_diameter_p_mean', 'label6'],
      dtype='object', length=117)
Index(['cap-diameter', 'stem-height', 'stem-width', 'cap-shape_b',
       'cap-shape_c', 'cap-shape_f', 'cap-shape_o', 'cap-shape_p',
       'cap-shape_s', 'cap-shape_x',
       ...
       'stem_width_gill_color_o_mean', 'label4',
       'cap_diameter_cap_color_e_min', 'cap_diameter_cap_color_e_max',
       'cap_diameter_cap_color_e_mean', 'label5', 'cap_diameter_p_min',
       'cap_diameter_p_max', 'cap_diameter_p_mean', 'label6'],
      dtype='object', length=117)
Index(['cap-diameter', 'stem-height', 'stem-width', 'cap-s

In [225]:
print(concatenated_df_test.shape)
last_column_test = concatenated_df_test.iloc[:, -1]
last_column_test = concatenated_df_test.iloc[:, -1].reset_index(drop=True)
print(last_column_test)

(18321, 111)
0        0
1        1
2        1
3        1
4        0
        ..
18316    0
18317    0
18318    0
18319    0
18320    1
Name: label, Length: 18321, dtype: int64


In [226]:
print(label_column_train)

common_index
0        1
1        0
2        0
3        1
4        0
        ..
42743    0
42744    1
42745    0
42746    0
42747    1
Name: label, Length: 42748, dtype: int64


In [227]:
print(top_features)

['cap-shape_s', 'gill-color_n', 'habitat_g', 'gill-attachment_x', 'gill-color_y', 'cap-shape_x', 'cap-surface_d', 'cap-color_y', 'cap-surface_y', 'cap-shape_b', 'gill-attachment_d', 'cap-shape_f', 'gill-color_w', 'cap-surface_s', 'cap-color_w', 'stem-color_y', 'gill-attachment_s', 'cap-shape_o', 'label', 'cap-surface_g', 'cap-color_n', 'stem-color_n', 'habitat_d', 'cap-surface_h', 'stem-color_w', 'gill-color_p', 'gill-attachment_a', 'cap-color_o', 'stem-width', 'gill-spacing_d', 'gill-attachment_e', 'gill-color_g', 'cap-color_g', 'habitat_m', 'season_w', 'does-bruise-or-bleed_t', 'does-bruise-or-bleed_f', 'season_u', 'stem-color_g', 'habitat_l', 'season_s', 'has-ring_f', 'has-ring_t', 'cap-surface_e', 'ring-type_e', 'season_a', 'cap-surface_i', 'ring-type_z', 'cap-surface_k', 'cap-surface_t']


In [239]:
# Get the indices of the columns in shuffled_data that correspond to the top 70 features
top_feature_indices_test = [concatenated_df_test.columns.get_loc(feature) for feature in top_features[:50]]

# Create a new dataframe that contains only the top 70 features
top_70_features_data_test = concatenated_df_test.iloc[:, top_feature_indices_test]

# Print the shape of the new dataframe
print(top_70_features_data_test.columns)

Index(['cap-shape_s', 'gill-color_n', 'habitat_g', 'gill-attachment_x',
       'gill-color_y', 'cap-shape_x', 'cap-surface_d', 'cap-color_y',
       'cap-surface_y', 'cap-shape_b', 'gill-attachment_d', 'cap-shape_f',
       'gill-color_w', 'cap-surface_s', 'cap-color_w', 'stem-color_y',
       'gill-attachment_s', 'cap-shape_o', 'label', 'cap-surface_g',
       'cap-color_n', 'stem-color_n', 'habitat_d', 'cap-surface_h',
       'stem-color_w', 'gill-color_p', 'gill-attachment_a', 'cap-color_o',
       'stem-width', 'gill-spacing_d', 'gill-attachment_e', 'gill-color_g',
       'cap-color_g', 'habitat_m', 'season_w', 'does-bruise-or-bleed_t',
       'does-bruise-or-bleed_f', 'season_u', 'stem-color_g', 'habitat_l',
       'season_s', 'has-ring_f', 'has-ring_t', 'cap-surface_e', 'ring-type_e',
       'season_a', 'cap-surface_i', 'ring-type_z', 'cap-surface_k',
       'cap-surface_t'],
      dtype='object')


In [237]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

X_train = top_70_features_data_train
y_train = label_column_train
#print(label_train.shape)

# Create a logistic regression object and fit it to the training data
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)

X_test = top_70_features_data_test

# Predict the class labels of the test data
y_pred = logreg.predict(X_test)

y_test = last_column_test

print(y_pred)
print(y_test)
accu = logreg.score(X_test, y_test)
print(accu)

[0 1 1 ... 0 0 1]
0        0
1        1
2        1
3        1
4        0
        ..
18316    0
18317    0
18318    0
18319    0
18320    1
Name: label, Length: 18321, dtype: int64
1.0


In [234]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, log_loss
import matplotlib.pyplot as plt

X_train = top_70_features_data
y_train = label_train

logreg = LogisticRegression(max_iter=1000)
losses = []
accuracies = []

for i in range(100):
    logreg.fit(X_train, y_train)
    y_pred = logreg.predict(shuffled_data_test)
    y_prob = logreg.predict_proba(shuffled_data_test)
    loss = log_loss(label_test, y_prob)
    accuracy = logreg.score(shuffled_data_test, label_test)
    losses.append(loss)
    accuracies.append(accuracy)

plt.plot(losses)
plt.title("Log loss")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.show()

plt.plot(accuracies)
plt.title("Accuracy")
plt.xlabel("Iteration")
plt.ylabel("Accuracy")
plt.show()

NameError: name 'top_70_features_data' is not defined

In [231]:
np.unique(label_test)

NameError: name 'label_test' is not defined

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

# Get the confusion matrix
cm = confusion_matrix(label_test, y_pred)

# Create a confusion matrix display object with labels
cm_display = ConfusionMatrixDisplay(cm, display_labels=[0,1])

# Plot the confusion matrix
cm_display.plot()